In [20]:
from sec_api import query_files, extract_item
import os

os.environ["POLARS_MAX_THREADS"] = "1"

import polars as pl

In [2]:
response = query_files(size=90)

In [3]:
df = pl.from_dicts(response["filings"])

In [4]:
df[["ticker", "companyName", "linkToFilingDetails"]].write_csv("filings.csv")
print("CSV file created successfully.")

CSV file created successfully.


In [2]:
df = pl.read_csv("filings.csv")
df

ticker,companyName,linkToFilingDetails
str,str,str
"""ARQ""","""Arq, Inc.""","""https://www.sec.gov/Archives/e…"
"""STOR""","""STORE CAPITAL LLC""","""https://www.sec.gov/Archives/e…"
"""PIAC""","""PRINCETON CAPITAL CORP""","""https://www.sec.gov/Archives/e…"
"""OXSQ""","""Oxford Square Capital Corp.""","""https://www.sec.gov/Archives/e…"
"""SEI""","""Solaris Energy Infrastructure,…","""https://www.sec.gov/Archives/e…"
…,…,…
"""SCM""","""Stellus Capital Investment Cor…","""https://www.sec.gov/Archives/e…"
"""JRVR""","""James River Group Holdings, Lt…","""https://www.sec.gov/Archives/e…"
"""NPCE""","""NeuroPace Inc""","""https://www.sec.gov/Archives/e…"


In [3]:
df = df.head(79)
df

ticker,companyName,linkToFilingDetails
str,str,str
"""ARQ""","""Arq, Inc.""","""https://www.sec.gov/Archives/e…"
"""STOR""","""STORE CAPITAL LLC""","""https://www.sec.gov/Archives/e…"
"""PIAC""","""PRINCETON CAPITAL CORP""","""https://www.sec.gov/Archives/e…"
"""OXSQ""","""Oxford Square Capital Corp.""","""https://www.sec.gov/Archives/e…"
"""SEI""","""Solaris Energy Infrastructure,…","""https://www.sec.gov/Archives/e…"
…,…,…
"""TBBK""","""Bancorp, Inc.""","""https://www.sec.gov/Archives/e…"
"""""","""Blue Owl Credit Income Corp.""","""https://www.sec.gov/Archives/e…"
"""PIF""","""North Haven Private Income Fun…","""https://www.sec.gov/Archives/e…"


In [ ]:
df = df.with_columns(
    [
        pl.when(pl.col("Risk Factor") == "")
        .then(pl.col("linkToFilingDetails").map_elements(extract_item))
        .otherwise(pl.col("Risk Factor"))
        .alias("Risk Factor")
    ]
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


ticker,companyName,linkToFilingDetails,Risk Factor
str,str,str,str
"""ARQ""","""Arq, Inc.""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors The f…"
"""STOR""","""STORE CAPITAL LLC""","""https://www.sec.gov/Archives/e…",""" Item 1A. RI SK FACTORS Ther…"
"""PIAC""","""PRINCETON CAPITAL CORP""","""https://www.sec.gov/Archives/e…",""" Item 1A. RISK FACTORS &#160…"
"""OXSQ""","""Oxford Square Capital Corp.""","""https://www.sec.gov/Archives/e…",""" Item 1A. &#160; Risk Factors …"
"""SEI""","""Solaris Energy Infrastructure,…","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors The fo…"
…,…,…,…
"""""","""IPALCO ENTERPRISES, INC.""","""https://www.sec.gov/Archives/e…",""" ITEM 1A. RISK FACTORS Inves…"
"""OKTA""","""Okta, Inc.""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors A des…"
"""TLGYF""","""TLGY ACQUISITION CORP""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors As a …"


In [24]:
df.filter(pl.col("Risk Factor") != "").write_csv("filings.csv")

In [27]:
df.write_parquet("raw_filings.parquet")

In [25]:
df.filter(pl.col("Risk Factor") != "").write_parquet("filings.parquet")

In [29]:
pl.read_parquet("filings.parquet")

ticker,companyName,linkToFilingDetails,Risk Factor
str,str,str,str
"""ARQ""","""Arq, Inc.""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors The f…"
"""STOR""","""STORE CAPITAL LLC""","""https://www.sec.gov/Archives/e…",""" Item 1A. RI SK FACTORS Ther…"
"""PIAC""","""PRINCETON CAPITAL CORP""","""https://www.sec.gov/Archives/e…",""" Item 1A. RISK FACTORS &#160…"
"""OXSQ""","""Oxford Square Capital Corp.""","""https://www.sec.gov/Archives/e…",""" Item 1A. &#160; Risk Factors …"
"""SEI""","""Solaris Energy Infrastructure,…","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors The fo…"
…,…,…,…
"""""","""IPALCO ENTERPRISES, INC.""","""https://www.sec.gov/Archives/e…",""" ITEM 1A. RISK FACTORS Inves…"
"""OKTA""","""Okta, Inc.""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors A des…"
"""TLGYF""","""TLGY ACQUISITION CORP""","""https://www.sec.gov/Archives/e…",""" Item 1A. Risk Factors As a …"


In [1]:
import fireducks.pandas as pd
from wordllama import WordLlama

wl = WordLlama.load()

In [20]:
df = pd.read_parquet("filings.parquet").rename(columns={"Risk Factor": "riskFactor"})
df["riskFactor"] = df["riskFactor"].str.lower()
df["riskFactor"] = df["riskFactor"].str.replace(
    r"item\s+1a\.?\s+[Rr][Ii]?\s*[Ss][Kk]\s*[Ff][Aa][Cc][Tt][Oo][Rr][Ss]?",
    "",
    regex=True,
)
df["riskFactor"] = df["riskFactor"].str.replace("item 1a risk factor", "", regex=False)
df["riskFactor"] = df["riskFactor"].str.replace("item 1a risk factors", "", regex=False)
df["riskFactor"] = df["riskFactor"].str.replace("risk factors", "", regex=False)
df["riskFactor"] = df["riskFactor"].str.replace(r"\s+", " ", regex=True)
df["riskFactor"] = df["riskFactor"].str.replace(r"[^\w\s]", "", regex=True)

df["riskFactor"] = df["riskFactor"].str.strip()

In [21]:
df

,ticker,companyName,linkToFilingDetails,riskFactor
0,ARQ,"Arq, Inc.",https://www.sec.gov/Archives/edgar/data/151515...,the following risks relate to us as of the dat...
1,STOR,STORE CAPITAL LLC,https://www.sec.gov/Archives/edgar/data/153899...,there are many factors that affect our busines...
2,PIAC,PRINCETON CAPITAL CORP,https://www.sec.gov/Archives/edgar/data/845385...,160 investing in our securities involves a num...
3,OXSQ,Oxford Square Capital Corp.,https://www.sec.gov/Archives/edgar/data/125942...,item 1a 160 investing in our securities involv...
4,SEI,"Solaris Energy Infrastructure, Inc.",https://www.sec.gov/Archives/edgar/data/169750...,the following are certain that affect our busi...
5,ASPC,ASPAC III Acquisition Corp.,https://www.sec.gov/Archives/edgar/data/189036...,160 as a smaller reporting company we are not ...
6,BYND,"BEYOND MEAT, INC.",https://www.sec.gov/Archives/edgar/data/165521...,risk factor summary we are providing the follo...
7,REI,"RING ENERGY, INC.",https://www.sec.gov/Archives/edgar/data/138419...,item 1a we are subject to various risks and un...
8,GWRS,"Global Water Resources, Inc.",https://www.sec.gov/Archives/edgar/data/143472...,in addition to the other information set forth...
9,MLR,MILLER INDUSTRIES INC /TN/,https://www.sec.gov/Archives/edgar/data/924822...,table_end 8203 in addition to information disc...


In [32]:
embeds = wl.embed(df["riskFactor"].tolist())
embeds = embeds.reshape(embeds.shape[0], -1, 256)
df["riskEmbedding"] = embeds.tolist()

In [36]:
df.head()

,ticker,companyName,linkToFilingDetails,riskFactor,riskEmbedding
0,ARQ,"Arq, Inc.",https://www.sec.gov/Archives/edgar/data/151515...,the following risks relate to us as of the dat...,"[[-0.0339742973446846, -0.11274787038564682, -..."
1,STOR,STORE CAPITAL LLC,https://www.sec.gov/Archives/edgar/data/153899...,there are many factors that affect our busines...,"[[0.010040835477411747, -0.10509602725505829, ..."
2,PIAC,PRINCETON CAPITAL CORP,https://www.sec.gov/Archives/edgar/data/845385...,160 investing in our securities involves a num...,"[[0.027260858565568924, -0.09319445490837097, ..."
3,OXSQ,Oxford Square Capital Corp.,https://www.sec.gov/Archives/edgar/data/125942...,item 1a 160 investing in our securities involv...,"[[0.00752898957580328, -0.10683222860097885, -..."
4,SEI,"Solaris Energy Infrastructure, Inc.",https://www.sec.gov/Archives/edgar/data/169750...,the following are certain that affect our busi...,"[[-0.029547376558184624, -0.1530650109052658, ..."


In [37]:
df.to_parquet("filings_embeddings.parquet")